In [2]:
import requests
import re
import time
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from wordcloud import WordCloud
%matplotlib inline

In [16]:
def get_reddit_data(subreddit, size, score, num_comments,after = None):

# create empty list for desired parameters
    title = []
    subreddit = []
    upvotes = []
    comments = []
    subscribers = []
    created_date = []
    url = []
    id = []

# create empty data frame of column names
    df = pd.DataFrame()

    # set after parameter to None (this is so we can start our scrape at a specific date in the for loop)
    #after = None
    headers = {'User-agent': 'Alex'}

    for a in range(20):                           # for loop to iterate through a range to 20
        print(f"Looping through page {a+1}")      # print a statement stating what page we are looping through
        if after == None:                         # if statement to check if loop needs to start from the beginning
            params = {                            # parameters that are necessary for scraping push shift api
            'subreddit': subreddit,           
            'after': '1531195200',                # Start date to scrape data (Epoch date in CST Monday, July 9, 2018 11:00:00 PM)
            'size': size,                         # 20 loops of 1000 should give us 20,000 posts
            'score': score,                        # upvote score
            'num_comments': num_comments                  # number of comments
            }
        else:
            params = {
            'subreddit': subreddit,
            'after': after,
            'size': size,
            'score': score,
            'num_comments': num_comments
            }
        link = 'https://api.pushshift.io/reddit/search/submission/'     # url to scrape push shift api
        res = requests.get(link, params=params, headers=headers)        # Get the json results based off of parameters

        if res.status_code == 200:                                      # if status code is a success continue through code
            the_json = res.json()
            for i in range(len(the_json['data'])):                      # Go through each loop that is in the json data
                        title.append(the_json['data'][i]['title'])      # append the empty lists with each desired parameter
                        subreddit.append(the_json['data'][i]['subreddit'])
                        comments.append(the_json['data'][i]['num_comments'])
                        upvotes.append(the_json['data'][i]['score'])
                        subscribers.append(the_json['data'][i]['subreddit_subscribers'])
                        created_date.append(the_json['data'][i]['created_utc'])
                        url.append(the_json['data'][i]['full_link'])
                        id.append(the_json['data'][i]['id'])
        else:
            print(res.status_code)
            break
        after = the_json['data'][i]['created_utc']     # make after equal the last posts epoch date so when the loop starts over it can start at this epoch date
        time.sleep(1)

# add the lists of each parameter to the df
df['title'] = title
df['subreddit'] = subreddit
df['comments'] = comments
df['upvotes'] = upvotes
df['subscribers'] = subscribers
df['created_date'] = created_date
df['url'] = url
df['id'] = id

In [22]:
get_reddit_data('thanosdidnothingwrong', size=1000, score='>0',num_comments= '>0')

Looping through page 1
Looping through page 2
414


In [9]:
df

,title,subreddit,comments,upvotes,subscribers,created_date,url,id
0,For those who fell... for balance,thanosdidnothingwrong,1,2,638343,1531195214,https://www.reddit.com/r/thanosdidnothingwrong...,8xla5n
1,Join here if banned or spared.,thanosdidnothingwrong,3,1,638334,1531195219,https://www.reddit.com/r/thanosdidnothingwrong...,8xla6c
2,Wow.,thanosdidnothingwrong,4,1,638275,1531195240,https://www.reddit.com/r/thanosdidnothingwrong...,8xlaah
3,I lived?,thanosdidnothingwrong,6,1,638248,1531195248,https://www.reddit.com/r/thanosdidnothingwrong...,8xlac2
4,Test,thanosdidnothingwrong,1,1,638207,1531195265,https://www.reddit.com/r/thanosdidnothingwrong...,8xlafc
5,This sub post-ban.,thanosdidnothingwrong,2,5,638200,1531195267,https://www.reddit.com/r/thanosdidnothingwrong...,8xlafz
6,"I look out on a sub after the great balance, a...",thanosdidnothingwrong,2,3,638135,1531195287,https://www.reddit.com/r/thanosdidnothingwrong...,8xlajj
7,BUT But but..,thanosdidnothingwrong,5,2,638131,1531195288,https://www.reddit.com/r/thanosdidnothingwrong...,8xlajt
8,Up vote this post if you're still here but you...,thanosdidnothingwrong,2,10,638127,1531195289,https://www.reddit.com/r/thanosdidnothingwrong...,8xlak4
9,So what's the kill count? Was anyone keeping t...,thanosdidnothingwrong,2,3,638123,1531195294,https://www.reddit.com/r/thanosdidnothingwrong...,8xlaks
